## Init Dependencies

In [1]:
import requests
import os
import pandas as pd
from datetime import datetime
import json
import numpy as np

### Workspace Setup

In [2]:
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

## Set Static Variables

In [3]:
URL = 'http://rsr.akvo.org/rest/v1/'
PROJECT_ID = '7950'
#PROJECT_ID = '7283'
#PROJECT_TYPE = 'grand_parent'
PROJECT_TYPE = 'parent'
#PROJECT_TYPE = 'child'
RSR_TOKEN = os.environ['RSR_TOKEN']
FMT = '/?format=json&limit=1'
FMT100 = '/?format=json&limit=100'

## Set Filter

In [4]:
#FILTER_DATE = '2017-01-01 - 2017-12-31'
FILTER_DATE = '2017'
FILTER_DATE_TYPE = 'yearly'
#FILTER_DATE_TYPE = 'semester'
FILTER_COUNTRY = ['Malawi','Mozambique','Zambia']
HTML_TYPE = 'table' #print or table

## Set Authentication

In [5]:
headers = {
    'content-type': 'application/json',
    'Authorization': RSR_TOKEN
}

## Helper Functions

In [6]:
def get_response(endpoint, param, value):
    uri = '{}{}{}&{}={}'.format(URL, endpoint, FMT100, param, value)
    print(get_time() + ' Fetching - ' + uri)
    data = requests.get(uri, headers=headers)
    data = data.json()
    return data

In [7]:
def get_time():
    now = datetime.now().time().strftime("%H:%M:%S")
    return now

In [8]:
def get_sibling_id(x):
    for k,v in x.items():
        return k

In [9]:
def get_report_type(ps,pe):
    rt = {'is_yearly':False}
    try:
        psm = ps.split('-')[1]
        pem = pe.split('-')[1]
        if psm == '01' and pem == '12':
            rt = {'is_yearly':True}
        if psm == '01' and pem == '01':
            rt = {'is_yearly':True}
    except:
        pass
    return rt

In [10]:
def get_dimension_country(dv):
    dp = dv['value'].split(' - ')
    dv = {}
    if dp[0].lower() in ['zambia','malawi','mozambique']:
        dv.update({
            'commodity':'',
            'country':dp[0],
            'has_commodity':False,
            'has_country':True
        })
    else:
        dv.update({
            'commodity':dp[0],
            'country':'',
            'has_commodity':False,
            'has_country':True
        })
    if len(dp) == 2:
        dv.update({
            'commodity':dp[0],
            'country':dp[1],
            'has_commodity':True,
            'has_country':True
        })
    return dv

## Find Related Project

In [11]:
related_project = get_response('related_project','related_project',PROJECT_ID)

15:47:05 Fetching - http://rsr.akvo.org/rest/v1/related_project/?format=json&limit=100&related_project=7950


In [12]:
results_framework_list = [PROJECT_ID]
if PROJECT_TYPE == 'parent':
    results_framework_list = list(pd.DataFrame(related_project['results'])['project'])
if PROJECT_TYPE == 'grand_parent':
    parent_list = list(pd.DataFrame(related_project['results'])['project'])
    results_framwork_list = []
    for second_level in parent_list:
        second_list = get_response('related_project','related_project',second_level)
        try:
            second_list = list(pd.DataFrame(second_list['results'])['project'])
            for third_level in second_list:
                results_framework_list.append(third_level)
        except:
            pass

In [13]:
results_framework_list

[8019, 7924, 7858]

### Trace All Children (Alternative)

In [14]:
def trace_all_childrens(project_id, all_childs, level):
    related = get_response('related_project','related_project',project_id)        
    if len(related['results']) > 0:
        for result in related['results']:
            level = level + 1
            print(level)
            all_childs.append(result)
            trace_all_childrens(result['project'], all_childs, level)
    return all_childs

### Trace All Children Results(Alternative)

In [15]:
def trace_all_results(all_childs):
    results_framework = []
    for child in all_childs:
        res = get_response('results_framework','project',child['project'])
        for rf in res['results']:
            results_framework.append(rf)
    return results_framework

### Concat Results Frameworks (Alternative)

In [16]:
def trace_onechildren():
    results_framework = []
    for i, rf in enumerate(results_framework_list):
        result_framework = get_response('results_framework','project',rf)['results']
        if i == 0:
            results_framework = result_framework
        else:
            for res in result_framework:
                results_framework.append(res)
    return results_framework

### Only Parents

In [17]:
results_framework = []
def no_trace():
    results_framework = get_response('results_framework','project',PROJECT_ID)['results']
    return results_framework

### Choose Trace Level

In [18]:
if PROJECT_TYPE == 'child':
    results_framework = no_trace()
if PROJECT_TYPE == 'parent':
    results_framework = trace_onechildren()
if PROJECT_TYPE == 'grand_parent':
    #all_childrens = trace_all_childrens(PROJECT_ID, [],1)
    results_framework = trace_onechildren()

15:47:06 Fetching - http://rsr.akvo.org/rest/v1/results_framework/?format=json&limit=100&project=8019
15:47:11 Fetching - http://rsr.akvo.org/rest/v1/results_framework/?format=json&limit=100&project=7924
15:47:16 Fetching - http://rsr.akvo.org/rest/v1/results_framework/?format=json&limit=100&project=7858


## Begin Transformations

In [19]:
results_framework = pd.DataFrame(results_framework)

### Generate List of All Objects

In [20]:
results_framework['child_projects'] = results_framework['child_projects'].apply(get_sibling_id)

In [21]:
if PROJECT_TYPE == 'grand_parent':
    results_framwork = results_framework[results_framework['child_projects'].isnull()].reset_index().drop(columns=['index'])

In [22]:
results_framework = results_framework.to_dict('records')

In [23]:
indicators = []
periods = []
dimension_names = []
dimension_values = []
disaggregations = []
disaggregation_targets = []
for result_framework in results_framework:
    rf_id = {'result':result_framework['id']}
    rf_project = {'project':result_framework['project_title']}
    rf_title = {'project_title':result_framework['title']}
    for indicator in result_framework['indicators']:
        indicator_id = indicator['id']
        indicator_type = int(indicator['measure'])
        indicator_title = {'indicator':indicator['title']}
        indicator.update({'indicator_type':indicator_type})
        for period in indicator['periods']:
            is_yearly = get_report_type(period['period_start'],period['period_end'])
            period.update(rf_title)
            period.update(rf_project)
            period.update(is_yearly)
            period.update(rf_id)
            period.update({'indicator':indicator_id})
            period.update({'indicator_title':indicator['title']})
            period.update({'indicator_type':indicator_type})
            periods.append(period)
            for data in period['data']:
                if len(data) > 0:
                    for disaggregation in data['disaggregations']:
                        disaggregation.update({'period':period['id']})
                        disaggregation.update({'parent_period':period['parent_period']})
                        disaggregation.update({'result_id':result_framework['id']})
                        disaggregation.update(rf_title)
                        disaggregation.update(rf_project)
                        disaggregation.update({'indicator_id':indicator_id})
                        disaggregation.update(indicator_title)
                        disaggregations.append(disaggregation)
            if len(period['disaggregation_targets']) > 0:
                for disaggregation_target in period['disaggregation_targets']:
                    disaggregation_target.update({'period':period['id']})
                    disaggregation_target.update({'parent_period':period['parent_period']})
                    disaggregation_target.update({'indicator_id':indicator_id})
                    disaggregation_target.update(indicator_title)
                    disaggregation_target.update({'result_id':result_framework['id']})
                    disaggregation_target.update(rf_title)
                    disaggregation_target.update(rf_project)
                    disaggregation_targets.append(disaggregation_target)
        #del indicator['periods']
        for dimension_name in indicator['dimension_names']:
            for dimension_value in dimension_name['values']:
                dimension_value.update(rf_id)
                dimension_update = get_dimension_country(dimension_value)
                dimension_value.update(dimension_update)
                dimension_values.append(dimension_value)
            # del dimension_name['values']
            dimension_name.update(rf_id)
            dimension_name.update({'indicator':indicator_id})
            dimension_names.append(dimension_name)
        #del indicator['dimension_names']
        indicators.append(indicator)

In [24]:
dimension_values = pd.DataFrame(dimension_values).groupby(['id']).first().reset_index()

## DN

In [25]:
dimension_names = pd.DataFrame(dimension_names).groupby(['id']).first()
dimension_names = dimension_names.drop(columns=['values','project','parent_dimension_name','result','indicator']).reset_index()
dimension_names = dimension_names.rename(columns={'id':'dimension_id','name':'dimension_name'})
dimension_values = dimension_values.merge(dimension_names,left_on='name',right_on='dimension_id',how='outer')

In [26]:
def fill_country(x):
    country = x['country']
    if country.lower() not in  ['zambia','malawi','mozambique']:
        country = x['project'].split(' ')[1]
    return country

### Disaggregation Values

In [27]:
remove_columns = [
    'created_at',
    'last_modified_at',
    'numerator',
    'denominator',
    'narrative',
    'dimension_value',
    'incomplete_data',
    'update',
    'dimension_name_disaggregation',
    'dimension_id'
]
rename_columns = {
    'value_dimension_values': 'disaggregation_name',
    'value_disaggregation': 'disaggregation_value',
    'dimension_name_dimension_values':'dimension_name'
}
fill_values = {
    'disaggregation_value':0,
    'incomplete_data':True
}
column_order = ['parent_dimension_value',
                'parent_period',
                'result',
                'country',
                'dimension_'
                'dimension_name',
                'disaggregation_name',
                'id',
                'commodity',
                'has_country',
                'has_commodity',
                'incomplete_data',
                'disaggregation_value',
                'period',
                'project_title',
                'project',
                'indicator',
                'indicator_id'
]
disaggregation_value = ['disaggregation_value']
disaggregations_merged = pd.DataFrame(disaggregations).drop(columns=['id']).merge(
    dimension_values, 
    how='outer', 
    left_on='dimension_value', 
    right_on='id', 
    suffixes=('_disaggregation','_dimension_values'))
disaggregations_merged = disaggregations_merged.drop(columns=remove_columns)
disaggregations_merged = disaggregations_merged.rename(columns=rename_columns)
disaggregations_merged = disaggregations_merged.fillna(value=fill_values)
disaggregations_merged['value'] = disaggregations_merged['disaggregation_value'].apply(lambda x:int(float(x)))
disaggregations_merged = disaggregations_merged.drop(columns=['disaggregation_value'])
disaggregations_merged = disaggregations_merged.dropna(subset=['parent_period'])
disaggregations_merged['parent_period'] = disaggregations_merged['parent_period'].astype(int)
disaggregations_merged['period'] = disaggregations_merged['period'].apply(lambda x: int(float(x)))
disaggregations_merged['indicator_id'] = disaggregations_merged['indicator_id'].apply(lambda x: int(float(x)))
disaggregations_merged['type'] = 'Cumulative Actual Values'
disaggregations_merged['country'] = disaggregations_merged.apply(fill_country , axis = 1)

#### Country R&D

In [28]:
if PROJECT_ID == '7282':
    disaggregations_merged['project'] = 'APPSA Zambia'
    disaggregations_merged['commodity'] = disaggregations_merged['commodity'].apply(lambda x: x.replace('-',' ').title())
    disaggregations_merged['commodity'] = disaggregations_merged['commodity'].apply(lambda x: "Country Project" if x == "" else x)
    disaggregations_merged['country'] = disaggregations_merged.apply(fill_country, axis=1)
    disaggregations_merged = disaggregations_merged.drop(columns=['project'])

### Target Values

In [29]:
targets = pd.DataFrame(disaggregation_targets).fillna(0).drop(columns=['id']).merge(
    dimension_values, 
    how='outer', 
    left_on='dimension_value', 
    right_on='id', 
    suffixes=('_target','_dimension_values'))
targets = targets.dropna(subset=['parent_period','id'])
targets['has_commodity'] = False
targets['has_country'] = True
fill_values = {
    'value_target':0
}
targets  = targets.fillna(value=fill_values)
targets['value'] =  targets['value_target'].apply(lambda x:int(float(x)))
targets = targets.drop(columns=['value_target'])
integer_list = ['id','name','parent_dimension_value','result','parent_period','dimension_value','dimension_id','period']
targets[integer_list] = targets[integer_list].astype(int)
rename_columns = {'value_dimension_values': 'disaggregation_name'}
unavailable_columns = ['disaggregation_value', 'incomplete_data','dimension_value','dimension_dimension_name']
column_order = [x for x in column_order if x not in unavailable_columns]
column_order = column_order + ['value','dimension_name','dimension_id']

In [30]:
targets = targets.rename(columns=rename_columns)[column_order]
targets['indicator_id'] = targets['indicator_id'].apply(lambda x: int(float(x)))
targets['type'] = 'Y4 RCoLs Targets'
if PROJECT_ID == '7282':
    targets['project'] = 'APPSA Zambia'
    targets['commodity'] = targets['commodity'].apply(lambda x: x.replace('-',' ').title())
    targets['commodity'] = targets['commodity'].apply(lambda x: "Country Project" if x == "" else x)
    targets['country'] = targets.apply(fill_country, axis=1)
    targets = targets.drop(columns=['project'])
else:
    targets['country'] = ''
    targets['country'] = targets.apply(fill_country, axis = 1)

In [31]:
disaggregations_merged = disaggregations_merged.rename(columns={'name':'dimension_id'})

In [32]:
order_columns = ['id','result','indicator_id','dimension_id','project_title','indicator','dimension_name','commodity','period','country','type','value']

In [33]:
targets = targets[order_columns]
disaggregations_merged = disaggregations_merged[order_columns]

## Ajax

### Redefining Period

In [34]:
periods_short = pd.DataFrame(periods)
periods_short = periods_short[['id','is_yearly','period_start','period_end','target_value','actual_value','indicator_type','indicator_title']]
periods_short['period_date'] = periods_short['period_start'] + ' - ' + periods_short['period_end']

### Defining Values

In [35]:
ajax = pd.concat([disaggregations_merged,targets],sort=False)
ajax = ajax.sort_values(by=['result','indicator_id','dimension_id','id'])
ajax = ajax.merge(periods_short,how='inner',left_on='period',right_on='id',suffixes=('_data','_period')).sort_values(['id_data','indicator_id','dimension_name'])
remove_columns = [
    'period_end',
    'period_start',
    'id_period',
    'period',
    'is_yearly'
]
ajax = ajax[pd.notnull(ajax['period_end'])]

In [36]:
ajax['cumulative'] = ajax['period_end'].apply(lambda x: True if x.split('-')[1] == '12' else False)
ajax = ajax[ajax['cumulative'] == True].drop(columns=['cumulative'])
ajax = ajax.drop(columns=remove_columns)
if FILTER_DATE_TYPE == 'yearly':
    ajax['year'] = ajax['period_date'].apply(lambda x: x.split(' - ')[0].split('-')[0])
    ajax = ajax[ajax['year'] == FILTER_DATE].drop(columns=['year','period_date'])
else:
    pass
    #ajax = ajax[ajax['period_date'] == FILTER_DATE].drop(columns=['period_date'])

In [37]:
def get_total_value(x,vtype):
    y = 0
    column_list = []
    if x['indicator_type'] == 1:
        y = x[vtype + '-MW'] + x[vtype + '-MZ'] + x[vtype + '-ZA']
    if x['indicator_type'] == 2:
        y = x[vtype + '-MW'] + x[vtype + '-MZ'] + x[vtype + '-ZA']
        y = round(y / 3,2)
    return y

In [38]:
order_columns = ['result',
                 'project_title',
                 'indicator_id',
                 'indicator_type',
                 'indicator',
                 'dimension_id',
                 'dimension_name',
                 'commodity',
                 'type',
                 'country',
                 'value',
                 'id_data']
ajax = ajax[order_columns]
ajax_group = ['result',
              'project_title',
              'indicator_id',
              'indicator_type',
              'indicator',
              'dimension_id',
              'dimension_name',
              'id_data',
              'commodity',
              'country',
              'type']
ajax_sort = ['result','indicator_id','dimension_id','id_data']
ajax = ajax.groupby(ajax_group).sum()
ajax = ajax.unstack('type').unstack('country').sort_values(ajax_sort)
ajax = ajax.groupby(level=[1,3,4,6,8],sort=False).sum().astype(int)
ajax = pd.DataFrame(ajax['value'].to_records())
ajax = ajax.rename(columns={
    "('Cumulative Actual Values', 'Malawi')": "CA-MW",
    "('Cumulative Actual Values', 'Mozambique')": "CA-MZ",
    "('Cumulative Actual Values', 'Zambia')": "CA-ZA",
    "('Y4 RCoLs Targets', 'Malawi')":"TG-MW",
    "('Y4 RCoLs Targets', 'Mozambique')":"TG-MZ",
    "('Y4 RCoLs Targets', 'Zambia')":"TG-ZA"
})
ajax['TG-TTL'] = ajax.apply(lambda x: get_total_value(x,'TG'), axis=1)
ajax['CA-TTL'] = ajax.apply(lambda x: get_total_value(x,'CA'), axis=1)

### Defining Attribute Values

In [39]:
array = pd.concat([disaggregations_merged,targets],sort=False)
array = array.sort_values(by=['result','indicator_id','dimension_id','id'])
array = array.merge(periods_short,how='inner',left_on='period',right_on='id',suffixes=('_data','_period')).sort_values(['id_data','indicator_id','dimension_name'])

In [40]:
array['variable'] = array.apply(lambda x: {
    'id':x['id_data'],
    'result':x['result'],
    'country':x['country'],
    'type':x['type'],
    'period':x['period'],
    'date':x['period_date'],
    'indicator_id':x['indicator_id'],
    'indicator_type':x['indicator_type'],
    'indicator_name':x['indicator'],
    'dimension':x['dimension_id'],
    'dimension_name':x['dimension_name'],
    'commodity':x['commodity'],
    'value': x['value'],
    'target_value':x['target_value'],
    'actual_value':x['actual_value']
},axis=1)
array = array[pd.notnull(array['period_end'])]

In [41]:
array['cumulative'] = array['period_end'].apply(lambda x: True if x.split('-')[1] == '12' else False)
array = array[array['cumulative'] == True].drop(columns=['cumulative'])
if FILTER_DATE_TYPE == 'yearly':
    array['year'] = array['period_date'].apply(lambda x: x.split(' - ')[0].split('-')[0])
    array = array[array['year'] == FILTER_DATE].drop(columns=['year','period_date'])
else:
    pass
    #ajax = ajax[ajax['period_date'] == FILTER_DATE].drop(columns=['period_date'])

In [42]:
array_group =  ['result','project_title','indicator_id','indicator_type','indicator','dimension_id','dimension_name','id_data','commodity','country','type']
array_sort = ['result','indicator_id','dimension_id','id_data']

In [43]:
def group_attribute(a):
    data = []
    for b in a:
        data.append(b)
    return(data)

In [44]:
array = array.groupby(array_group)['variable'].apply(group_attribute).reset_index()
array = array.groupby(array_group).first().unstack('type').unstack('country').sort_values(array_sort)

In [45]:
array = array.groupby(level=[1,4,6,8],sort=False).first()
array = pd.DataFrame(array['variable'].to_records())
array = array.rename(columns={
    "('Cumulative Actual Values', 'Malawi')": "CA-MW-D",
    "('Cumulative Actual Values', 'Mozambique')": "CA-MZ-D",
    "('Cumulative Actual Values', 'Zambia')": "CA-ZA-D",
    "('Y4 RCoLs Targets', 'Malawi')":"TG-MW-D",
    "('Y4 RCoLs Targets', 'Mozambique')":"TG-MZ-D",
    "('Y4 RCoLs Targets', 'Zambia')":"TG-ZA-D"
})

In [47]:
array = array.drop(columns=['project_title','indicator','commodity','dimension_name'])
array = pd.merge(ajax, array, left_index=True, right_index=True)

In [48]:
array = array.replace({np.nan: None})

In [49]:
project = list(pd.DataFrame(results_framework).groupby('title').first().reset_index()['title'])

In [50]:
indicator_title = list(pd.DataFrame(indicators).groupby('title').first().reset_index()['title'])

In [51]:
indicator_var = pd.DataFrame(indicators).groupby('title').first().reset_index()[['title','id','description']]

In [60]:
array['TG-MW-D'] = array['TG-MW-D'].fillna(array['CA-MW-D'])
array['TG-MZ-D'] = array['TG-MZ-D'].fillna(array['CA-MZ-D'])
array['TG-ZA-D'] = array['TG-ZA-D'].fillna(array['CA-ZA-D'])
array['CA-MW-D'] = array['CA-MW-D'].fillna(array['TG-MW-D'])
array['CA-MZ-D'] = array['CA-MZ-D'].fillna(array['TG-MZ-D'])
array['CA-ZA-D'] = array['CA-ZA-D'].fillna(array['TG-ZA-D'])

In [61]:
array

,project_title,indicator_type,indicator,dimension_name,commodity,CA-MW,CA-MZ,CA-ZA,TG-MW,TG-MZ,TG-ZA,TG-TTL,CA-TTL,CA-MW-D,CA-MZ-D,CA-ZA-D,TG-MW-D,TG-MZ-D,TG-ZA-D
0,Project Development Objective: Increase the av...,1,PDO 1 Number of technologies that are being ma...,Type of technology,Number of improved seed varieties,55,51,43,55,23,66,144.00,149.00,"[{'id': 1385, 'result': 37006, 'country': 'Mal...","[{'id': 1291, 'result': 36998, 'country': 'Moz...","[{'id': 1338, 'result': 37002, 'country': 'Zam...","[{'id': 1385, 'result': 37006, 'country': 'Mal...","[{'id': 1385, 'result': 37006, 'country': 'Mal...","[{'id': 1338, 'result': 37002, 'country': 'Zam..."
1,Project Development Objective: Increase the av...,1,PDO 1 Number of technologies that are being ma...,Type of technology,Maize,12,12,9,24,5,17,46.00,33.00,"[{'id': 1386, 'result': 37006, 'country': 'Mal...","[{'id': 1292, 'result': 36998, 'country': 'Moz...","[{'id': 1339, 'result': 37002, 'country': 'Zam...","[{'id': 1386, 'result': 37006, 'country': 'Mal...","[{'id': 1386, 'result': 37006, 'country': 'Mal...","[{'id': 1339, 'result': 37002, 'country': 'Zam..."
2,Project Development Objective: Increase the av...,1,PDO 1 Number of technologies that are being ma...,Type of technology,Rice,9,10,4,8,6,6,20.00,23.00,"[{'id': 1387, 'result': 37006, 'country': 'Mal...","[{'id': 1293, 'result': 36998, 'country': 'Moz...","[{'id': 1340, 'result': 37002, 'country': 'Zam...","[{'id': 1387, 'result': 37006, 'country': 'Mal...","[{'id': 1387, 'result': 37006, 'country': 'Mal...","[{'id': 1340, 'result': 37002, 'country': 'Zam..."
3,Project Development Objective: Increase the av...,1,PDO 1 Number of technologies that are being ma...,Type of technology,Legumes,34,23,22,23,12,43,78.00,79.00,"[{'id': 1388, 'result': 37006, 'country': 'Mal...","[{'id': 1294, 'result': 36998, 'country': 'Moz...","[{'id': 1341, 'result': 37002, 'country': 'Zam...","[{'id': 1388, 'result': 37006, 'country': 'Mal...","[{'id': 1388, 'result': 37006, 'country': 'Mal...","[{'id': 1341, 'result': 37002, 'country': 'Zam..."
4,Project Development Objective: Increase the av...,1,PDO 1 Number of technologies that are being ma...,Type of technology,Cassava,0,6,7,0,0,0,0.00,13.00,"[{'id': 1389, 'result': 37006, 'country': 'Mal...","[{'id': 1295, 'result': 36998, 'country': 'Moz...","[{'id': 1342, 'result': 37002, 'country': 'Zam...","[{'id': 1389, 'result': 37006, 'country': 'Mal...","[{'id': 1389, 'result': 37006, 'country': 'Mal...","[{'id': 1342, 'result': 37002, 'country': 'Zam..."
5,Project Development Objective: Increase the av...,1,PDO 1 Number of technologies that are being ma...,Type of technology,"Number of improved agronomic, pest and disease...",15,19,15,17,2,12,31.00,49.00,"[{'id': 1390, 'result': 37006, 'country': 'Mal...","[{'id': 1296, 'result': 36998, 'country': 'Moz...","[{'id': 1343, 'result': 37002, 'country': 'Zam...","[{'id': 1390, 'result': 37006, 'country': 'Mal...","[{'id': 1390, 'result': 37006, 'country': 'Mal...","[{'id': 1343, 'result': 37002, 'country': 'Zam..."
6,Project Development Objective: Increase the av...,1,PDO 1 Number of technologies that are being ma...,Type of technology,"Number of improved post-harvest storage, labor...",2,45,33,4,2,10,16.00,80.00,"[{'id': 1391, 'result': 37006, 'country': 'Mal...","[{'id': 1297, 'result': 36998, 'country': 'Moz...","[{'id': 1344, 'result': 37002, 'country': 'Zam...","[{'id': 1391, 'result': 37006, 'country': 'Mal...","[{'id': 1391, 'result': 37006, 'country': 'Mal...","[{'id': 1344, 'result': 37002, 'country': 'Zam..."
7,Project Development Objective: Increase the av...,2,PDO 2 Proportion of Lead Farmers in targeted a...,Gender RCoL,Male,0,0,0,27,52,45,41.33,0.00,"[{'id': 1402, 'result': 37006, 'country': 'Mal...","[{'id': 1402, 'result': 37006, 'country': 'Mal...","[{'id': 1355, 'result': 37002, 'country': 'Zam...","[{'id': 1402, 'result': 37006, 'country': 'Mal...","[{'id': 1402, 'result': 37006, 'country': 'Mal...","[{'id': 1355, 'result': 3

## Generate Results

### Merge Result Period

In [ ]:
resutls = pd.concat([disaggregations_merged,targets],sort=False)
resutls = resutls.sort_values(by=['indicator_id','dimension_name','id'])

In [ ]:
resutls['commodity'] = '• ' + resutls['commodity']
resutls['indicator'] = '+ ' + resutls['indicator']
resutls['project_title'] = '### ' + resutls['project_title']

In [ ]:
results = resutls.merge(periods_short, how='inner', left_on='period', right_on='id', suffixes=('_data','_period'))
remove_columns = [
    'id_data',
    'period_end',
    'period_start',
    'id_period',
    'indicator_id',
    'id_period',
    'period',
    'dimension_name'
]
results = results.drop(columns=remove_columns)

In [ ]:
results

## Filter Parameters

### Filter Date

In [ ]:
results = results[results['period_date'] == FILTER_DATE]

### Filter Country

In [ ]:
results = results[results['country'].isin(FILTER_COUNTRY)]

### Set Grouping

In [ ]:
group_table = ['project_title','indicator','commodity','country','type']

In [ ]:
results = results.drop(columns=['period_date','is_yearly'])

In [ ]:
original = resutls.groupby(['project_title',
                            'indicator',
                            'indicator_id',
                            'commodity',
                            'dimension_name',
                            'country',
                            'type']).first()

In [ ]:
original = original.drop(columns=['id','period','result','dimension_id']).sort_values(by=[
    'project_title',
    'indicator_id',
    'dimension_name'])

In [ ]:
results = results.groupby(group_table).first().unstack('type').unstack('country').fillna(0)

In [ ]:
indicator_sum = results.sum(level=[0,1])
indicator_sum = indicator_sum.stack().stack().reset_index().rename(columns={0:'value'}).dropna()
indicator_sum['commodity'] = ''
results = results.stack().stack().reset_index()
results = results.append(indicator_sum, sort='False')
results = results.groupby(group_table).first().unstack('type').unstack('country').fillna(0)

In [ ]:
results = results.astype(int)

In [ ]:
project_title = results.sum(level=[0])
project_title = project_title.stack().stack().reset_index()
project_title['value'] = ''
project_title['indicator'] = ''
project_title['commodity'] = ''
results = results.stack().stack().reset_index()
results = results.append(project_title, sort='False')

In [ ]:
results = results.drop(columns='result').groupby(group_table).first().unstack('type').unstack('country').fillna(0)

In [ ]:
html_output = 'file_name.html'
results.to_html(html_output)

## Beautify HTML

In [ ]:
from bs4 import BeautifulSoup as bs

In [ ]:
variable_name = 'PDO Level Results Indicators'

In [ ]:
with open(html_output) as htm:
    html = htm.read()
    soup = bs(html)

In [ ]:
soup.find('table')['border'] = 0
soup.find('table')['class'] = "table"

In [ ]:
def remove_all_attrs_except(sp):
    whitelist = ['border','table']
    blacklist = ['project_title','indicator','commodity','value','country']
    header = ['Cumulative Actual Values','Y4 RCoLs Targets']
    country = ['Malawi','Zambia','Mozambique']
    for tag in sp.find_all(True):
        if tag.name == 'table':
            tag['id'] = 'rsrtable'
        if tag.name not in whitelist:
            tag.attrs = {}
        if tag.name == 'th':
            if '•' in tag.text:
                text = str(tag.text).replace('•','')
                tag.string.replace_with(bs(text))
                tag['style']='padding-left:50'
            if '+' in tag.text:
                text = str(tag.text).replace('+','')
                tag.string.replace_with(bs(text))
                tag['style']='padding-left:30'
            if '###' in tag.text:
                text = str(tag.text).replace('###','')
                tag.string.replace_with(bs(text))
                if HTML_TYPE == 'print':
                    tag['colspan'] = 7
                else:
                    tag.decompose()
        if tag.text == '0':
            tag.string.replace_with('-')
            tag['class'] = 'text-right'
        if tag.text == '':
            tag.decompose()
        if tag.text in blacklist:
            tag.decompose()
        if tag.text == 'type':
            tag.string.replace_with(bs(variable_name))
            tag['rowspan'] = 2
        if tag.text in header:
            tag['colspan'] = 3
            tag['class'] = 'text-center'
        if tag.text in country:
            tag['class'] = 'text-center'
        if tag.text == "actual_value":
            tag.insert_before(soup.new_tag("th"))
        try:
            float(tag.text)
            tag['class'] = 'text-right'
        except:
            pass
    return sp

In [ ]:
soup = remove_all_attrs_except(soup)

In [ ]:
new_head = soup.new_tag("head")
soup.html.append(new_head)

css = soup.new_tag("link", 
                   rel="stylesheet", 
                   href="https://maxcdn.bootstrapcdn.com/bootstrap/4.0.0/css/bootstrap.min.css", 
                   integrity="sha384-Gn5384xqQ1aoWXA+058RXPxPg6fy4IWvTNh0E263XmFcJlSAwiGgFAW/dAiS6JXm",
                   crossorigin="anonymous")
datatable_css = soup.new_tag("link", 
                   rel="stylesheet", 
                   href="https://cdn.datatables.net/v/bs4/jq-3.3.1/dt-1.10.18/b-1.5.6/b-flash-1.5.6/fh-3.1.4/r-2.2.2/rg-1.1.0/datatables.min.css" 
)
soup.head.append(css)
soup.head.append(datatable_css)

In [ ]:
table = bs(str(soup.body.table))
soup.html.body.decompose()

In [ ]:
for i, tr in enumerate(table.html.body.table.find_all('tr')):
    if tr.contents == ['\n']:
        tr.decompose()

In [ ]:
new_body = soup.new_tag("body")
soup.html.append(new_body)
soup.body.append(table.html.body.table)

In [ ]:
datatable_js = soup.new_tag("script", 
                   type="text/javascript", 
                   src="https://cdn.datatables.net/v/bs4/jq-3.3.1/dt-1.10.18/b-1.5.6/b-flash-1.5.6/fh-3.1.4/r-2.2.2/rg-1.1.0/datatables.min.js" 
)

In [ ]:
custom_js = soup.new_tag("script", 
                   type="text/javascript", 
                   src="/table.js" 
)

In [ ]:
soup.head.append(datatable_js)
soup.body.append(custom_js)

In [ ]:
with open("file_name_edit.html", "w") as outf:
    outf.write(str(soup))